In [1]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

Below is code with a link to a happy or sad dataset which contains 80 images, 40 happy and 40 sad. 
Create a convolutional neural network that trains to 100% accuracy on these images,  which cancels training upon hitting training accuracy of >.999

Hint -- it will work best with 3 convolutional layers.

In [1]:
import tensorflow as tf
import os
import zipfile


DESIRED_ACCURACY = 0.999

!wget --no-check-certificate \
    "https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip" \
    -O "/tmp/happy-or-sad.zip"

zip_ref = zipfile.ZipFile("/tmp/happy-or-sad.zip", 'r')
zip_ref.extractall("/tmp/h-or-s")
zip_ref.close()

class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('accuracy')>0.999):
      print("\nReached 99.9% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()

--2022-05-07 08:09:28--  https://storage.googleapis.com/laurencemoroney-blog.appspot.com/happy-or-sad.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 64.233.191.128, 173.194.192.128, 209.85.146.128, ...
Connecting to storage.googleapis.com (storage.googleapis.com)|64.233.191.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2670333 (2.5M) [application/zip]
Saving to: ‘/tmp/happy-or-sad.zip’

/tmp/happy-or-sad.z 100%[===================>]   2.55M  --.-KB/s    in 0.02s   

2022-05-07 08:09:28 (161 MB/s) - ‘/tmp/happy-or-sad.zip’ saved [2670333/2670333]



In [8]:
# This Code Block should Define and Compile the Model
model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, (3,3), activation='relu', 
                           input_shape=(150, 150, 3)),
    tf.keras.layers.MaxPooling2D(2, 2),
    tf.keras.layers.Conv2D(64, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Conv2D(128, (3,3), activation='relu'),
    tf.keras.layers.MaxPooling2D(2,2),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(512, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

from tensorflow.keras.optimizers import RMSprop

# Each file is 150x150 dimension

model.compile(loss='binary_crossentropy',
              optimizer=RMSprop(learning_rate=0.001),
              metrics=['accuracy'])

In [9]:
# This code block should create an instance of an ImageDataGenerator called train_datagen 
# And a train_generator by calling train_datagen.flow_from_directory

from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1/255)

train_generator = train_datagen.flow_from_directory(
        '/tmp/h-or-s/',  # This is the source directory for training images
        target_size=(150, 150),  # All images will be resized to 150x150
        batch_size=10,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

# Expected output: 'Found 80 images belonging to 2 classes'

Found 80 images belonging to 2 classes.


In [13]:
# This code block should call model.fit and train for
# a number of epochs. 
history = model.fit(
          train_generator,
      epochs=15,
      steps_per_epoch=4,
      verbose=1,callbacks=[callbacks])
    
# Expected output: "Reached 99.9% accuracy so cancelling training!""

Epoch 1/15
3/4 [=====================>........] - ETA: 0s - loss: 0.0120 - accuracy: 1.0000
Reached 99.9% accuracy so cancelling training!
4/4 [==============================] - 0s 47ms/step - loss: 0.0101 - accuracy: 1.0000


In [16]:
# Identify emoji from an online downloaded image
import numpy as np
from google.colab import files
from keras.preprocessing import image

uploaded = files.upload()

for fn in uploaded.keys():
 
  # predicting images
  path = '/content/' + fn
  img = image.load_img(path, target_size=(150, 150))
  x = image.img_to_array(img)
  x = x / 255
  print(x[0])
  print("Hello -",x.shape)
  x = np.expand_dims(x, axis=0)
  print(x[0])
  print('Hello - ',x.shape)
  #images = np.vstack([x])
  #print(images[0])
  classes = model.predict(x, batch_size=10)
  print(classes[0])
  if classes[0]>0.5:
    print(fn + " is Sad")
  else:
    print(fn + " is Happy")

Saving boy-sad-icon-teenage-boy-icons.jpg to boy-sad-icon-teenage-boy-icons.jpg
[[1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [0.99215686 0.99215686 0.99215686]
 [0.99607843 0.99607843 0.99607843]
 [0.9254902  0.9254902  0.9254902 ]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [0.93333334 0.93333334 0.93333334]
 [1.         1.         1.        ]
 [1.         1.         1.        ]
 [1.         1.     